In [ ]:
#!pip install geetools

In [ ]:
import math 
import pprint

In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=O6cyPySi88GC-frL5TgtBg1K5r4o_SkceyHJa4F0tJE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6t0I-CfLD7Fobs2myS2Nt5EQ1NUBx71DIN0SzTzZX3mjfNGsSxY58

Successfully saved authorization token.


In [ ]:
iniDate = '2021-04-01'
endDate = '2021-04-30'


In [ ]:
exportBand_evapo = ee.List(["PET", "ET"])
exportBand_img_smos = ee.List(["ssm", "susm"])
#.filter(ee.Filter.date('2015-01-01', '2017-12-31'))
#geometry =  ee.FeatureCollection('users/alexanderquevedo/limite_frankenstein_utm'); 
geometry = ee.Geometry.Rectangle([-105.70671, 18.91105, -101.45436, 22.75942])


In [ ]:
def getQABits(image, start, end, newName):
    #Compute the bits we need to extract.
    p = 0
    for i in range(start,(end+1)):
        p += math.pow(2, i)

    # Return a single band image of the extracted QA bits, giving the band
    # a new name.
    return image.select([0], [newName])\
                  .bitwiseAnd(p)\
                  .rightShift(start)

#A function to mask out cloudy pixels.
def maskClouds(img):
  # Select the QA band.
    QA = img.select('DetailedQA')
  # Get the MOD_LAND_QA bits
    internalCloud = getQABits(QA, 0, 1, 'MOD_LAND_QA')
  # Return an image masking out cloudy areas.
    return img.mask(internalCloud.eq(0))

def maskBadData(image):
    invalid = image.select('DetailedQA').bitwiseAnd(0x6).neq(0)
    clean = image.mask(invalid.Not())
    return(clean)

def maskSummaryQA(img):
    QA = img.select('SummaryQA').eq(0)
    best = img.mask(QA)
    return(best)

# function to add system time band
def addTimeBand(image):
    return image.addBands(image.metadata('system:time_start').rename(["time"]))


In [ ]:
#### Terreno

dem = ee.Image('USGS/SRTMGL1_003')
slope_t = ee.Terrain.slope(dem)
#terreno  = dem.addBands(slope_t)
#### MODIS 
img_ndvi = ee.ImageCollection('MODIS/006/MOD13Q1').map(maskSummaryQA).select('NDVI').filterDate(iniDate,endDate).mean()
img_evapo = ee.ImageCollection('MODIS/006/MOD16A2').select(exportBand_evapo).filterDate(iniDate,endDate).mean()
img_temp = ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Day_1km').filterDate(iniDate,endDate).mean()

#### SMOS
img_smos =  ee.ImageCollection('NASA_USDA/HSL/soil_moisture').select(exportBand_img_smos).filterDate(iniDate,endDate).mean()

### Precipitacion

img_precipitacion = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filterDate(iniDate,endDate).sum()

print('Image:', str(img_precipitacion.getInfo())+'\n')

#img_smos = ee.ImageCollection('NASA_USDA/HSL/soil_moisture').filterDate(iniDate,endDate).mean()


Image: {'type': 'Image', 'bands': []}



In [ ]:
#https://stackoverflow.com/questions/57547601/imagecollection-reduce-functions-producing-single-pixel-image-when-exported-us

task_dem = ee.batch.Export.image.toDrive(**{
  'image': dem,
  'description': 'dem',
  'scale': 250,
  'crs': 'EPSG:32613',
  'folder': 'incendios_2021',
  'region': geometry.getInfo()['coordinates']
})

task_slope = ee.batch.Export.image.toDrive(**{
  'image': slope_t,
  'description': 'slope',
  'scale': 250,
  'crs': 'EPSG:32613',
  'folder': 'incendios_2021',
  'region': geometry.getInfo()['coordinates']
})

task_nvdi = ee.batch.Export.image.toDrive(**{
  'image': img_ndvi,
  'description': 'NDVI',
  'scale': 250,
  'crs': 'EPSG:32613',
  'folder': 'incendios_2021',
  'region': geometry.getInfo()['coordinates']
})

task_evapo = ee.batch.Export.image.toDrive(**{
  'image': img_evapo,
  'description': 'evapotranspiracion',
  'scale': 250,
  'crs': 'EPSG:32613',
  'folder': 'incendios_2021',
  'region': geometry.getInfo()['coordinates']
})

task_temp = ee.batch.Export.image.toDrive(**{
  'image': img_temp,
  'description': 'temperatura',
  'scale': 250,
  'crs': 'EPSG:32613',
  'folder': 'incendios_2021',
  'region': geometry.getInfo()['coordinates']
})

task_smos = ee.batch.Export.image.toDrive(**{
  'image': img_smos,
  'description': 'smos',
  'scale': 250,
  'crs': 'EPSG:32613',
  'folder': 'incendios_2021',
  'region': geometry.getInfo()['coordinates']
})

task_prep = ee.batch.Export.image.toDrive(**{
  'image': img_precipitacion,
  'description': 'precipitacion',
  'scale': 250,
  'crs': 'EPSG:32613',
  'folder': 'incendios_2021',
  'region': geometry.getInfo()['coordinates']
})

In [ ]:
task_dem.start()
task_slope.start()
task_nvdi.start()
task_evapo.start()
task_temp.start()
task_smos.start()
task_prep.start()

In [ ]:
#task_dem.start()

In [ ]:
#task_nvdi.status